In [1]:
import os

In [2]:
from nilearn.input_data import NiftiMasker
from nilearn.plotting import plot_roi
from nilearn.image.image import mean_img

In [3]:
# As this is raw resting-state EPI, the background is noisy and we cannot
# rely on the 'background' masking strategy. We need to use the 'epi' one
nifti_masker = NiftiMasker(standardize=True, mask_strategy='epi',
                           memory="nilearn_cache", memory_level=2,
                           smoothing_fwhm=8)


In [4]:
filename = os.getcwd() + "/dataset/train/Patient_01/Patient_01.nii.gz"

In [5]:
nifti_masker.fit(filename)
mask_img = nifti_masker.mask_img_

In [6]:
# calculate mean image for the background
mean_func_img = mean_img(filename)

In [15]:
plot_roi(mask_img, mean_func_img, display_mode='y', cut_coords=4, title="Mask")


In [11]:
# Preprocess data with the NiftiMasker

nifti_masker.fit(filename)
fmri_masked = nifti_masker.transform(filename)
# fmri_masked is now a 2D matrix, (n_voxels x n_time_points)

In [13]:
from sklearn.decomposition import FastICA
n_components = 9
ica = FastICA(n_components=n_components, random_state=42)
components_masked = ica.fit_transform(fmri_masked.T).T

/Users/rc/Documents/Acads/1819/180/MP2/segthor/sklearn/decomposition/fastica_.py:303: UserWarning: n_components is too large: it will be set to 1
  warnings.warn('n_components is too large: it will be set to %s' % n_components)


In [16]:
components = nifti_masker.inverse_transform(components_masked)

# Visualize results
from nilearn.plotting import plot_stat_map, show
from nilearn.image import index_img

plot_stat_map(index_img(components, 0), mean_func_img,
              display_mode='y', cut_coords=4, title="Component 0")

show()

/Users/rc/Documents/Acads/1819/180/MP2/segthor/nilearn/input_data/base_masker.py:224: UserWarning: Persisting input arguments took 1.63s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  img = self._cache(masking.unmask)(X, self.mask_img_)
